In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
import matplotlib.pyplot as plt
from numpy import log10, sqrt
import glob


mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def obs_pred_rsquare(obs, pred):
    # Determines the prop of variability in a data set accounted for by a model
    # In other words, this determines the proportion of variation explained by
    # the 1:1 line in an observed-predicted plot.
    return 1 - sum((obs - pred) ** 2) / sum((obs - np.mean(obs)) ** 2)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# Part 1: Load files

## Load engineered HCRIS file

In [2]:
hcris_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/HCRIS_data/FilteredEngineeredPUF_p5.pkl')
hcris_df.sort_values(by='E_A_HOS_C1_7099', ascending=True, inplace=True)
print(hcris_df.shape)

##########  Convert negative (erroneous) HCRIS HAC penalty values to positive values ##########
hcris_df['E_A_HOS_C1_7099'] = np.abs(hcris_df['E_A_HOS_C1_7099'])

hcris_df.drop(labels=['S2_1_C1_35', 'S2_1_C1_40', 'S2_1_C2_40', 
                     'E_A_HOS_C1_7096', 'E_A_HOS_C1_7097', 'E_A_HOS_C1_7093',
                     'E_A_HOS_C1_7090', 'E_A_HOS_C1_7094', 'E_A_HOS_C1_7091' ,
                     'E_A_HOS_C1_49', 'E_A_HOS_C1_50' ,'E_A_HOS_C1_54',
                     'E_A_HOS_C1_68', 'E_A_HOS_C1_93', 'E_A_HOS_C1_47', 'E_A_HOS_C1_48',
                     #'S2_1_C2_2', #'E_A_HOS_C1_72',
                     'NPI', 'RPT_STUS_CD', 
                     'FY_BGN_DT', 'FY_END_DT',
                     'PROC_DT', 'INITL_RPT_SW', 
                     'LAST_RPT_SW', 'TRNSMTL_NUM', 
                     'FI_NUM', 'ADR_VNDR_CD', 'FI_CREAT_DT', 
                     'UTIL_CD', 'NPR_DT', 'SPEC_IND', 'FI_RCPT_DT',
                    ], axis=1, inplace=True)

hcris_df.sort_values(by=['PRVDR_NUM', 'FILE_YEAR'], inplace=True)
hcris_df.head(6)


(40706, 44)


,RPT_REC_NUM,S2_1_C2_2,E_A_HOS_C1_72,E_A_HOS_C1_59,E_A_HOS_C1_7099,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,FILE_YEAR,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59
13685,724514,AL,70348911.0,75649340.0,NaN,9,010001,2015,75649340.0,75382394.0,753824.0,74628570.0,756493.4
15563,644872,AL,65062506.0,78945166.0,NaN,9,010001,2016,78945166.0,78378493.0,783785.0,77594708.0,789451.66
7913,696465,AL,73140841.0,79371374.0,788866.0,9,010001,2017,79371374.0,78886607.0,788866.0,78097741.0,793713.74
25349,685213,AL,68535114.0,76368818.0,NaN,9,010001,2018,75782340.0,75479726.0,754797.0,74724929.0,763688.18
30859,708691,AL,68068016.0,78286207.0,NaN,9,010001,2019,77819374.0,76848116.0,768481.0,76079635.0,782862.07
36720,713432,AL,68496274.0,77838336.0,NaN,9,010001,2020,76997773.0,75688676.0,756887.0,74931789.0,778383.36


## Load Rand cost report file

In [3]:
mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'

rand_df = pd.read_csv(mydir + "data/Rand_CostReport/rand_hcris_free_2022_11_01.csv", 
                    encoding='latin-1',
                    converters={
                                "Hospital cost report record number (with leading zeros) [rpt_rec_num]":str,  
                                "Medicare inpatient payments under prospective payment system [mdcr_inpat_pps_pymts]":str,
                                "Cost reporting period beginning date [FY_BGN_DT]":str,
                               },)

var_ = ["Hospital cost report record number (with leading zeros) [rpt_rec_num]",
        "Medicare inpatient payments under prospective payment system [mdcr_inpat_pps_pymts]"]

rand_df = rand_df.filter(items=var_, axis=1)
rand_df.rename(columns={
        "Medicare provider number [prvdr_num]": "PRVDR_NUM",
        "Hospital cost report record number (with leading zeros) [rpt_rec_num]": "RPT_REC_NUM",
        "Medicare inpatient payments under prospective payment system [mdcr_inpat_pps_pymts]": "mdcr_inpat_pps_pymts",
        }, inplace=True)

rand_df['mdcr_inpat_pps_pymts'] = pd.to_numeric(rand_df['mdcr_inpat_pps_pymts'], errors='coerce')

print(rand_df.shape)
rand_df.head()


(4624, 2)


,RPT_REC_NUM,mdcr_inpat_pps_pymts
0,675143,573504.94
1,682853,1328801.27
2,686501,1462371.50
3,686676,2053268.78
4,686893,NaN


## Load merged HAC/HAI data files containing penalty assignments and reproduced penalty assignments and imputed SIR denominators

In [4]:
### MODIFY THIS

hac_hai_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/HAI_HAC_2015.pkl')
print(hac_hai_df.shape)

ls = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
for yr in ls:
    tdf = pd.read_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/HAI_HAC_' + yr + '.pkl')
    hac_hai_df = pd.concat([hac_hai_df, tdf])
    print(hac_hai_df.shape)
    
print(sorted(list(hac_hai_df)))

print(hac_hai_df['file_month'].unique())
hac_hai_df.head()

(3336, 34)
(6688, 42)
(10002, 59)
(13302, 76)
(16576, 76)
(19794, 78)
(22998, 78)
(26153, 78)
['AHRQ PSI-90 Footnote', 'AHRQ PSI-90 Score', 'CAUTI Footnote', 'CAUTI Observed Cases', 'CAUTI Predicted Cases', 'CAUTI Score', 'CAUTI Urinary Catheter Days', 'CAUTI W Z Score', 'CAUTI delta', 'CAUTI derived SIR', 'CAUTI derived W Z Score', 'CAUTI derived Winsorized SIR', 'CAUTI derived score', 'CDI Footnote', 'CDI Observed Cases', 'CDI Predicted Cases', 'CDI Score', 'CDI W Z Score', 'CDI delta', 'CDI derived SIR', 'CDI derived W Z Score', 'CDI derived Winsorized SIR', 'CDI derived score', 'CDI patient days', 'CLABSI Device Days', 'CLABSI Footnote', 'CLABSI Observed Cases', 'CLABSI Predicted Cases', 'CLABSI Score', 'CLABSI W Z Score', 'CLABSI delta', 'CLABSI derived SIR', 'CLABSI derived W Z Score', 'CLABSI derived Winsorized SIR', 'CLABSI derived score', 'Domain 1 End Date', 'Domain 1 Footnote', 'Domain 1 Score', 'Domain 1 Score Footnote', 'Domain 1 Start Date', 'Domain 2 Footnote', 'Domain 2

,AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility ID,Facility Name,Fiscal Year,State,Total HAC Score,file_month,file_year,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date
0,1.0,9.0,7.0,2013-06-30 00:00:00,1.0,2011-07-01 00:00:00,2013-12-31,8.0,2012-01-01,110064,MIDTOWN MEDICAL CENTER,2015,GA,5.550,10,2015,No,27213.0,13259.0,13954.0,40.0,19.0,22.953,30.015,1.742692,0.633017,9.0,7.0,5.550,No,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,NaN,NaN,2013-06-30 00:00:00,5.0,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,390323,ADVANCED SURGICAL HOSPITAL,2015,PA,5.000,10,2015,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,5.000,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.0,4.0,7.0,2013-06-30 00:00:00,8.0,2011-07-01 00:00:00,2013-12-31,5.5,2012-01-01,490141,SPOTSYLVANIA REGIONAL MEDICAL CENTER,2015,VA,6.375,10,2015,No,2751.0,1600.0,1151.0,1.0,1.0,2.081,1.728,0.480538,0.578704,4.0,7.0,6.375,No,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,2013-06-30 00:00:00,4.0,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,670066,"BASIN HEALTHCARE CENTER, LLC",2015,TX,4.000,10,2015,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,4.000,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,NaN,NaN,2013-06-30 00:00:00,2.0,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,670067,BAYLOR ORTHOPEDIC AND SPINE HOSPITAL AT ARLINGTON,2015,TX,2.000,10,2015,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,2.000,No,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tdf = hac_hai_df[hac_hai_df['State'] == 'MD']
print(tdf.shape)
tdf.head()

print(tdf['file_year'].unique())

(373, 78)
['2015' '2016' '2017' '2018' '2019' '2020' '2021' '2022']


In [6]:
tot_pen = hac_hai_df.shape[0]
ls = hac_hai_df['Payment Reduction Reproduced?'].tolist()
num_correct = ls.count(1)
print(100 * num_correct/tot_pen)

print(hac_hai_df['file_month'].unique())


96.84930983061217
['10' '11' '04']


## Merge HCRIS and RAND dataframes

In [7]:
hcris_rand_df = hcris_df.merge(rand_df, how='outer', on=['RPT_REC_NUM'])
prvdrs = hac_hai_df['Facility ID'].unique()

## Filter out providers that aren't in the HAI_HAC dataframe
prvdrs = list(set(prvdrs))
hcris_rand_df = hcris_rand_df[hcris_rand_df["PRVDR_NUM"].isin(prvdrs)]

print('hcris_rand_df.shape:', hcris_rand_df.shape)
print(len(hcris_rand_df['PRVDR_NUM'].unique()), 'unique CMS numbers in hcris_rand file')
hcris_rand_df.sort_values(by='mdcr_inpat_pps_pymts', ascending=False, inplace=True)



#tdf = hcris_rand_df[hcris_rand_df['PRVDR_NUM'] == '140119']
hcris_rand_df.head(10)

hcris_rand_df.shape: (21935, 14)
3437 unique CMS numbers in hcris_rand file


,RPT_REC_NUM,S2_1_C2_2,E_A_HOS_C1_72,E_A_HOS_C1_59,E_A_HOS_C1_7099,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,FILE_YEAR,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,mdcr_inpat_pps_pymts
24395,711561,NY,808802623.0,849677262.0,NaN,2,330214,2020,763550708.0,765197446.0,7651974.0,757545472.0,8496772.62,966293248.0
24033,696215,NY,841705775.0,861869618.0,7823406.0,2,330101,2020,784623441.0,782340652.0,7823407.0,774517245.0,8618696.18,861869201.9
3939,724972,CA,535972206.0,570295018.0,3962512.0,2,050625,2020,529297953.0,529651688.0,5296517.0,524355171.0,5702950.18,570294784.0
23825,722632,NY,477669491.0,519537799.0,NaN,2,330024,2021,450022301.0,449810952.0,4498110.0,445312842.0,5195377.99,519537664.0
16885,713938,MA,489327906.0,514105740.0,NaN,2,220071,2021,478162736.0,478844449.0,4788444.0,474056005.0,5141057.4,514105600.0
18483,718437,MN,508826576.0,509213574.0,NaN,12,240010,2021,463417050.0,466826020.0,4668260.0,462157760.0,5092135.74,509213440.0
3581,711358,CA,471105850.0,487989421.0,371519.0,2,050441,2020,451889752.0,452792639.0,4527926.0,448264713.0,4879894.21,487989248.0
6316,720381,FL,433149637.0,477165142.0,NaN,2,100007,2021,439391363.0,437258255.0,4372583.0,432885672.0,4771651.42,477165056.0
23928,722769,NY,456773037.0,473922266.0,3881550.0,2,330059,2021,389901150.0,388154974.0,3881550.0,384273424.0,4739222.66,473922048.0
3593,706949,CA,449790946.0,453043821.0,4154543.0,10,050454,2020,416090005.0,415454316.0,4154543.0,411299773.0,4530438.21,453043712.0


### 

## Merge the HAI/HAC dataframe with the HCRIS/RAND dataframe

In [8]:
print(hac_hai_df['file_month'].unique())


['10' '11' '04']


In [9]:
hcris_rand_df.rename(columns={
        'PRVDR_NUM': 'Facility ID',
        'FILE_YEAR': 'file_year',
        }, inplace=True)

main_df = hcris_rand_df.merge(hac_hai_df, how='outer', on=['Facility ID', 'file_year'])

main_df = main_df[~main_df['file_month'].isin([np.nan, float('NaN')])]

col = main_df.pop('E_A_HOS_C1_7099')
main_df.insert(main_df.shape[1] - 4, 'E_A_HOS_C1_7099', col)

main_df.sort_values(by='E_A_HOS_C1_7099', ascending=True, inplace=True)
main_df.sort_values(by=['Facility ID', 'file_year'], inplace=True, ascending=False)

############### Impute HAC penalties based on RAND data #########################
mdcr_inpat_pps_pymts = main_df['mdcr_inpat_pps_pymts'].tolist()
pen_a = main_df['Payment Reduction'].tolist()

pen = []
for i, val in enumerate(mdcr_inpat_pps_pymts):
    
    if pen_a[i] == 'Yes':
        p = 0.01 * (val/0.99)
        pen.append(np.round(p, 0))
        
    elif pen_a[i] == 'No':
        p = 0.01 * val
        pen.append(np.round(p, 0))
    
    else:
        pen.append(np.nan)

main_df['HAC penalty (imputed from RAND)'] = pen
#################################################################################

print('main_df.shape:', main_df.shape)
print(len(main_df['Facility ID'].unique()), 'unique CMS numbers in main dataframe')

print(main_df['file_year'].unique())
print(main_df['file_month'].unique())
    
main_df['file date'] = main_df['file_year'].astype(str) + '_' + main_df['file_month'].astype(str)
print(main_df['file date'].unique())

main_df.sort_values(by=['Facility ID', 'file_year'], inplace=True, ascending=True)
main_df.head(10)

main_df.shape: (26466, 91)
3472 unique CMS numbers in main dataframe
['2022' '2021' '2020' '2019' '2018' '2017' '2016' '2015']
['04' '10' '11']
['2022_04' '2021_10' '2020_10' '2019_10' '2018_10' '2017_10' '2016_11'
 '2015_10']


,RPT_REC_NUM,S2_1_C2_2,E_A_HOS_C1_72,E_A_HOS_C1_59,PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,mdcr_inpat_pps_pymts,AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,E_A_HOS_C1_7099,CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date
3167,724514,AL,70348911.0,75649340.0,9,010001,2015,75649340.0,75382394.0,753824.0,74628570.0,756493.4,NaN,3.0,9.0,9.0,2013-06-30 00:00:00,3.0000,2011-07-01 00:00:00,2013-12-31,9.0000,2012-01-01,SOUTHEAST ALABAMA MEDICAL CENTER,2015.0,AL,6.9000,10,No,22394.0,16196.0,6198.0,33.0,9.0,19.435,9.300,1.697968,0.967742,9.0,9.0,6.900000,No,1.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015_10
3168,644872,AL,65062506.0,78945166.0,9,010001,2016,78945166.0,78378493.0,783785.0,77594708.0,789451.66,NaN,1.0,10.0,10.0,6302014.0,1.0000,7012012.0,2014-12-31,8.0000,2013-01-01,SOUTHEAST ALABAMA MEDICAL CENTER,2016.0,AL,6.2500,11,No,19927.0,14812.0,5115.0,56.0,17.0,17.783,7.675,3.149075,2.214984,10.0,10.0,6.250000,No,1.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016_11
3169,696465,AL,73140841.0,79371374.0,9,010001,2017,79371374.0,78886607.0,788866.0,78097741.0,793713.74,NaN,1.0,10.0,10.0,6302015.0,1.0000,7012013.0,2015-12-31,8.0000,2014-01-01,SOUTHEAST ALABAMA MEDICAL CENTER,2017.0,AL,6.9500,10,Yes,18260.0,13811.0,4449.0,50.0,18.0,16.583,6.676,3.015136,2.696225,10.0,10.0,6.610000,Yes,1.0,0.340000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,7.0,NaN,6.0,182743.0,176802.0,12.0,94.0,14.625,118.846,0.820513,0.79094,6.0,5.0,2.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,788866.0,NaN,NaN,NaN,NaN,NaN,2017_10
3170,685213,AL,68535114.0,76368818.0,9,010001,2018,75782340.0,75479726.0,754797.0,74724929.0,763688.18,NaN,NaN,NaN,NaN,09/30/2015,-1.2795,07/01/2014,2016-12-31,0.4826,2015-01-01,SOUTHEAST ALABAMA MEDICAL CENTER,2018.0,AL,0.2182,10,No,40996.0,26729.0,14267.0,40.0,21.0,31.194,14.917,1.282300,1.407800,NaN,NaN,0.301583,No,1.0,-0.083383,-0.318253,-0.291262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181535.0,178892.0,11.0,62.0,12.076,130.103,0.910900,0.47650,NaN,NaN,-0.123860,0.243189,NaN,1.3847,-1.0689,1.6280,NaN,NaN,0.3071,NaN,-1.2795,0.1619,1.2823,1.702953,1.4078,1.919262,0.9109,0.063911,NaN,0.4765,-0.9

In [10]:
tdf = main_df[main_df['State'] == 'MD']
print(tdf.shape)
tdf.head()

print(tdf['file_year'].unique())

(376, 92)
['2015' '2016' '2017' '2018' '2019' '2020' '2021' '2022']


## Ensure proper state abbreviations

In [11]:
PRVDR_NUM = main_df['Facility ID'].tolist()
prvdrs = []
for p in PRVDR_NUM:
    prvdrs.append(p[0:2])

main_df['PRVDR_STATE_CODE'] = prvdrs
main_df['S2_1_C2_2'].fillna('no state', inplace=True)
S2_1_C2_2 = main_df['S2_1_C2_2']

PRVDRS = []
states = []
ls = []
for i, state in enumerate(S2_1_C2_2):
    p = PRVDR_NUM[i]
    if state == 'no state':
        tdf = main_df[main_df['PRVDR_STATE_CODE'] == p[0:2]]
        tdf = tdf[~tdf['S2_1_C2_2'].isin([np.nan, float('NaN'), 'no state'])]
        s2 = tdf['S2_1_C2_2'].unique()
        if len(s2) > 1:
            break
            
        else:
            states.append(s2[0])
    else:
        states.append(state)
        
main_df['STATE'] = states
main_df.drop(labels=['PRVDR_STATE_CODE', 'S2_1_C2_2'], axis=1, inplace=True)

###     

## Calculate differences between reported and imputed penalty amounts

In [12]:
###########  Calculate differences between reported, imputed, and inferred amounts  ##########

O1 = np.array(main_df['E_A_HOS_C1_59'].tolist())
O2 = np.array(main_df['mdcr_inpat_pps_pymts'].tolist())
main_df['Dollar difference in payments (RAND vs E_A_HOS_C1_59)'] = np.abs(O1 - O2)
main_df['% Difference in payments (RAND vs E_A_HOS_C1_59)'] = 100 * np.abs(O1 - O2) / ((O1 + O2)/2)

T = np.round(np.array(main_df['E_A_HOS_C1_7099'].tolist()),0)
O = np.round(np.array(main_df['HAC penalty (imputed from RAND)'].tolist()),0)
main_df['% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND)'] = 100 * np.abs(O - T) / (T)
main_df['Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND)'] = np.abs(O - T)

O = np.round(np.array(main_df['Reconstructed HAC penalty'].tolist()),0)
main_df['% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty)'] = 100 * np.abs(O - T) / (T)
main_df['Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty)'] = np.abs(O - T)

main_df.sort_values(by='E_A_HOS_C1_7099', ascending=True, inplace=True)

main_df.head()

,RPT_REC_NUM,E_A_HOS_C1_72,E_A_HOS_C1_59,PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,mdcr_inpat_pps_pymts,AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,E_A_HOS_C1_7099,CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty)
3153,699112,76191.0,62273.0,2,450219,2020,62273.0,77792.0,778.0,77014.0,622.73,62272.98,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31,NaN,2017-01-01,BAYLOR SCOTT & WHITE MEDICAL CENTER - LLANO,2020.0,TX,1.3968,10,Yes,150.0,133.0,17.0,0.0,0.0,0.065,0.009,0.0,0.0,NaN,NaN,1.396800,Yes,1.0,0.000000,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,5.0,NaN,1398.0,1398.0,0.0,0.0,0.027,0.417,0.0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,NaN,NaN,06/30/2018,07/01/2016,629.0,2020_10,TX,0.02,0.000032,1086.792453,576.0,1367.924528,725.0
16323,645789,327887.0,322008.0,4,370030,2016,322008.0,394349.0,3943.0,390406.0,3220.08,NaN,8.0,NaN,NaN,6302014.0,8.0000,7012012.0,2014-12-31,NaN,2013-01-01,BLACKWELL REGIONAL HOSPITAL,2016.0,OK,8.0000,11,Yes,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,8.000000,Yes,1.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,NaN,NaN,NaN,NaN,NaN,2016_11,OK,NaN,NaN,NaN,NaN,6583.050847,3884.0
18653,725518,5475.0,NaN,6,440231,2018,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,09/30/2015,-0.0071,07/01/2014,2016-12-31,1.0381,2015-01-01,TRUSTPOINT HOSPITAL,2018.0,TN,0.8814,10,Yes,773.0,327.0,446.0,0.0,0.0,0.400,0.463,0.0,0.0,NaN,NaN,-0.605141,Yes,1.0,1.486541,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,13353.0,13353.0,0.0,3.0,0.455,5.328,0.0,0.5631,NaN,NaN,1.748778,NaN,NaN,NaN,1.0381,NaN,NaN,NaN,NaN,NaN,-0.0071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,0.5631,-0.710678,NaN,NaN,NaN,2018_10,TN,NaN,NaN,NaN,NaN,NaN,NaN
20558,632292,4965.0,7542.0,10,490104,2015,7542.0,7542.0,75.0,7467.0,75.42,NaN,NaN,NaN,NaN,2013-06-30 00:00:00,NaN,2011-07-01 00:00:00,2013-12-31,NaN,2012-01-01,HIRAM W DAVIS MEDICAL CENTER,2015.0,VA,NaN,10,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

-----

In [13]:
main_df['avg_Reconstructed HAC penalty'] = main_df.groupby(['Facility ID'])['Reconstructed HAC penalty'].transform('mean')



In [14]:

tdf = main_df.filter(items=['Facility ID', 'file_year', 'Reconstructed HAC penalty', 'avg_Reconstructed HAC penalty'],  axis=1)
tdf = tdf[~tdf['Reconstructed HAC penalty'].isin([np.nan, float('NaN')])]
tdf = tdf[~tdf['avg_Reconstructed HAC penalty'].isin([np.nan, float('NaN')])]

obs = np.log10(tdf['Reconstructed HAC penalty']) # obs
pred = np.log10(tdf['avg_Reconstructed HAC penalty']) # pred

#slope, intercept, r, p, se = stats.linregress(x, y)

#print('slope:', np.round(slope, 2))
#print('intercept:', np.round(intercept, 2))
#print('r-square:', np.round(r**2, 2))

print(obs_pred_rsquare(obs, pred))
plt.scatter(obs, pred)
#plt.plot(x, intercept + slope*x, 'r', label='fitted line')
plt.xlabel('avg_Reconstructed HAC penalty' + ', log10')
plt.ylabel('Reconstructed HAC penalty' + ', log10')
plt.show()

0.9663651940447895


# Part 4: Create a patched HAC penalty feature

**Reason:** Not all rows have usable values for: 
1. The reconstructed E_A_HOS_C1_7099 amount 
2. The reported E_A_HOS_C1_7099 amount
3. The amount that can be derived from E_A_HOS_C1_59
4. Average hospital-specific penalty value

In [15]:
p_fill = np.round(100 * main_df['E_A_HOS_C1_7099'].notna().sum()/main_df.shape[0], 2)
print('E_A_HOS_C1_7099 values obtained for only', p_fill, '% of rows.\n')

p_fill = np.round(100 * main_df['Reconstructed HAC penalty'].notna().sum()/main_df.shape[0], 2)
print('Reconstructed HAC penalty values obtained for', p_fill, '% of rows.\n')

p_fill = np.round(100 * main_df['HAC penalty imputed from E_A_HOS_C1_59'].notna().sum()/main_df.shape[0], 2)
print('Values for HAC penalty imputed from E_A_HOS_C1_59 obtained for', p_fill, '% of rows.\n')

p_fill = np.round(100 * main_df['avg_Reconstructed HAC penalty'].notna().sum()/main_df.shape[0], 2)
print('avg_Reconstructed HAC penalty values obtained for', p_fill, '% of rows.\n')


E_A_HOS_C1_7099 values obtained for only 24.47 % of rows.

Reconstructed HAC penalty values obtained for 81.56 % of rows.

Values for HAC penalty imputed from E_A_HOS_C1_59 obtained for 81.63 % of rows.

avg_Reconstructed HAC penalty values obtained for 99.32 % of rows.



**Solution:** Prioritize reconstructed E_A_HOS_C1_7099 amounts and try to fill missing values with reported E_A_HOS_C1_7099 amounts. Then, try to fill remaining missing values with amounts derived from E_A_HOS_C1_59. If that fails, use the statistically inferred hypothetical HAC penalty amounts.

In [16]:
order = ['Reconstructed HAC penalty',
         'E_A_HOS_C1_7099',
         'HAC penalty imputed from E_A_HOS_C1_59',
         'avg_Reconstructed HAC penalty',
        ]

main_df['HAC penalty, final'] = main_df[order[0]]
        
main_df['HAC penalty, final'] = main_df['HAC penalty, final'].fillna(main_df[order[1]])
p_fill = np.round(100 * main_df['HAC penalty, final'].notna().sum()/main_df.shape[0], 2)
print('patched HAC penalty values obtained for', p_fill, '% of rows')

main_df['HAC penalty, final'] = main_df['HAC penalty, final'].fillna(main_df[order[2]])
p_fill = np.round(100 * main_df['HAC penalty, final'].notna().sum()/main_df.shape[0], 2)
print('patched HAC penalty values obtained for', p_fill, '% of rows')

main_df['HAC penalty, final'] = main_df['HAC penalty, final'].fillna(main_df[order[3]])
p_fill = np.round(100 * main_df['HAC penalty, final'].notna().sum()/main_df.shape[0], 2)
print('patched HAC penalty values obtained for', p_fill, '% of rows')


patched HAC penalty values obtained for 81.56 % of rows
patched HAC penalty values obtained for 81.63 % of rows
patched HAC penalty values obtained for 99.33 % of rows


#      

# Part 4: Final processing and save to file

In [17]:
print(main_df['file_year'].unique())
print(main_df['file_month'].unique())
    
main_df['file date'] = main_df['file_year'].astype(str) + '_' + main_df['file_month'].astype(str)
print(main_df['file date'].unique())


['2020' '2016' '2018' '2015' '2017' '2019' '2021' '2022']
['10' '11' '04']
['2020_10' '2016_11' '2018_10' '2015_10' '2017_10' '2019_10' '2021_10'
 '2022_04']


In [18]:
print(main_df.shape)

main_df.rename(columns={'E_A_HOS_C1_72': 'IPPS interim payment (E_A_HOS_C1_72)', 
                        'E_A_HOS_C1_59': 'IPPS payment (E_A_HOS_C1_59)', 
                        'E_A_HOS_C1_7099': 'HAC penalty (E_A_HOS_C1_7099)',
                        'mdcr_inpat_pps_pymts': 'IPPS payment (from RAND)',
                        }, inplace=True)

features = ['Line 19', 'Reconstructed IPPS payment (pre HAC penalty)', 'Reconstructed HAC penalty', 
            'Reconstructed IPPS payment (post HAC penalty)', 'HAC penalty imputed from E_A_HOS_C1_59',
            'HAC penalty (E_A_HOS_C1_7099)', 'HAC penalty, final']

for f in features:
    main_df[f] = main_df[f].fillna(0)
    main_df[f] = main_df.groupby(['Facility ID', 'file_year'])[f].transform(sum)

main_df.sort_values(by=['Facility ID', 'file_year', 'Line 19'], inplace=True, ascending=False)
main_df.drop_duplicates(subset=['Facility ID', 'file_year'], inplace=True, keep='first')

print(main_df.shape)

main_df.head()

(26466, 100)
(26153, 100)


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final"
26635,NaN,NaN,NaN,NaN,670266,2022,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2019-01-01,TOWNSEN MEMORIAL HOSPITAL,2022.0,TX,NaN,04,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,12/31/2019,07/01/2018,NaN,2022_04,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
26634,NaN,NaN,NaN,NaN,670260,2022,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2019-01-01,TEXAS HEALTH HOSPITAL FRISCO,2022.0,TX,NaN,04,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,12/31/2019,07/01/2018,NaN,2022_04,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
26633,NaN,NaN,NaN,NaN,670259,2022,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2019-01-01,CAPROCK HOSPITAL,2022.0,TX,NaN,04,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,12/31/2019,07/01/2018,NaN,2022_04,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
26632,NaN,NaN,NaN,NaN,670143,2022,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2019-01-01,ASCENSION SETON BASTROP,2022.0,TX,NaN,04,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,12/31/2019,07/01/2018,NaN,2022_04,TX,NaN

In [19]:
main_df.to_csv('~/GitHub/HACRP-HAIs/data/Compiled_HCRIS-HACRP-HAI-RAND/Compiled_HCRIS-HACRP-HAI-RAND.csv', index=False)
main_df.to_pickle('~/GitHub/HACRP-HAIs/data/Compiled_HCRIS-HACRP-HAI-RAND/Compiled_HCRIS-HACRP-HAI-RAND.pkl', protocol=5)


In [20]:
tdf = main_df[main_df['Facility ID'] == '140119']
print(tdf.shape)
tdf.head(tdf.shape[0])

(8, 100)


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final"
24950,NaN,NaN,NaN,NaN,140119,2022,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2019-01-01,RUSH UNIVERSITY MEDICAL CENTER,2022.0,IL,-0.5083,04,No,41480.0,12145.0,29335.0,9.0,14.0,19.986,33.458,0.450300,0.418400,NaN,NaN,-0.508821,No,1.0,0.000521,0.000107,-0.000234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161796.0,142270.0,5.0,53.0,7.755,116.182,0.644700,0.45620,NaN,NaN,0.002407,0.000944,NaN,-0.5120,-0.2608,-0.4684,NaN,NaN,-0.2454,NaN,-1.4951,-0.0680,0.4503,-0.512107,0.4184,-0.468166,0.6447,-0.246344,0.0,0.4562,-0.263207,12/31/2019,07/01/2018,NaN,2022_04,IL,NaN,NaN,NaN,NaN,NaN,NaN,2.083921e+06,2.083921e+06
22992,NaN,NaN,NaN,NaN,140119,2021,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31,NaN,2018-01-01,RUSH UNIVERSITY MEDICAL CENTER,2021.0,IL,-0.2108,10,No,82104.0,23653.0,58451.0,22.0,48.0,37.925,65.807,0.580100,0.729400,NaN,NaN,-0.210421,No,1.0,-0.000379,-0.001954,-0.001803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,321436.0,286556.0,16.0,122.0,16.070,229.773,0.995600,0.53100,NaN,NaN,0.001292,0.000192,NaN,-0.3377,-0.2335,0.1204,NaN,NaN,0.3678,23.0,-1.5047,0.3229,0.5801,-0.335746,0.7294,0.122203,0.9956,0.367608,0.0,0.5310,-0.234792,06/30/2019,07/01/2017,NaN,2021_10,IL,NaN,NaN,NaN,NaN,NaN,NaN,2.083921e+06,2.083921e+06
51,709126,243113458.0,245254730.0,2,140119,2020,224618257.0,224474608.0,2244746.0,222229862.0,2452547.30,245254656.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31,NaN,2017-01-01,RUSH UNIVERSITY MEDICAL CENTER,2020.0,IL,0.0055,10,No,79176.0,21901.0,57275.0,20.0,61.0,35.328,65.349,0.566100,0.933400,NaN,NaN,0.005727,No,1.0,-0.000227,0.001822,-0.002896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313809.0,282425.0,19.0,200.0,17.359,226.856,1.094500,0.88160,NaN,NaN,0.001198,-0.001584,NaN,-0.4598,0.5105,0.4099,NaN,NaN,0.4927,NaN,-1.4781,0.5577,0.